In [14]:
import os
import shutil
import codecs
import chardet

def run_txt(input_dir):
    run = 1
    for file in os.listdir(input_dir):
        os.rename(f'./defect/{file}', f'./defect/defect{run}.txt')
        run += 1

def detect(filepath):
    with open(filepath, 'rb') as file:
        print(chardet.detect(file.read()))

detect('./defect/defect1.txt')

def run_tojson(input_dir, output_json):
    """
    takes txt files and copies dictionary to json file
    """
    for file in os.listdir(input_dir):
        with codecs.open(file, 'r', encoding='ascii') as input_file:
            with codecs.open(output_json, 'w', encoding='utf-8') as output_file:
                shutil.copyfileobj(input_file, output_file)

run_tojson('./defect/', './defect_runs.json')




{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 3131: ordinal not in range(128)